In [ ]:
from datetime import datetime, timedelta
from collections import Counter
import json

class Bike:
    count = 1

    def __init__(self, bike_type):
        self.bike_id = Bike.count
        Bike.count += 1
        self.type = bike_type
        self.status = 'dostępny'
        self.current_station = None

    def change_status(self, new_status):
        self.status = new_status

    def assign_to_station(self, station):
        self.current_station = station


class Station:
    count = 1

    def __init__(self, name, location, capacity):
        self.station_id = Station.count
        Station.count += 1
        self.name = name
        self.location = location
        self.capacity = capacity
        self.bikes = []

    def add_bike(self, bike):
        if len(self.bikes) >= self.capacity:
            raise Exception("Stacja jest pełna.")
        self.bikes.append(bike)
        bike.assign_to_station(self)
        bike.change_status('dostępny')

    def remove_bike(self, bike):
        if bike not in self.bikes:
            raise Exception("Nie ma tego roweru na tej stacji.")
        self.bikes.remove(bike)
        bike.assign_to_station(None)
        bike.change_status('niedostępny')

    def get_available_bikes(self):
        return [bike for bike in self.bikes if bike.status == 'dostępny']

    def print_available_bikes(self):
        available = self.get_available_bikes()
        if not available:
            print("Brak dostępnych rowerów.")
            return
        print("Dostępne rowery:")
        for bike in available:
            print(f"ID: {bike.bike_id}, Typ: {bike.type}")


class Rental:
    count = 1

    def __init__(self, user, bike, start_station):
        self.rental_id = Rental.count
        Rental.count += 1
        self.user = user
        self.bike = bike
        self.start_station = start_station
        self.start_time = datetime.now()
        self.end_station = None
        self.end_time = None
        self.active = True

    def end_rental(self, station):
        self.end_station = station
        station.add_bike(self.bike)
        self.end_time = datetime.now()
        self.active = False
        self.bike.change_status('dostępny')


class Reservation:
    count = 1

    def __init__(self, user, station, bike_type, reserved_at, expiration_time):
        self.reservation_id = Reservation.count
        Reservation.count += 1
        self.user = user
        self.station = station
        self.bike_type = bike_type
        self.reserved_at = reserved_at
        self.expiration_time = expiration_time
        self.fulfilled = False

    def fulfill(self):
        self.fulfilled = True

    def is_active(self):
        return not self.fulfilled and datetime.now() < self.expiration_time
    def __str__(self):
        return f"Rezerwacja ID {self.reservation_id}, rower: {self.bike_type}, stacja: {self.station.name}"

class Repair:

    def __init__(self, bike, zglaszajacy):
        self.bike = bike
        self.user = zglaszajacy
        self.start_time = datetime.now()
        self.duration = timedelta(minutes=2)
        self.finished = False
        self.completed_at = None

        bike.change_status('w naprawie')
        bike.assign_to_station(None)


    def is_done(self):
        return datetime.now() >= self.start_time + self.duration

    def complete(self, station):
        if self.finished:
            return
        if self.is_done():
            self.bike.change_status('dostępny')
            station.add_bike(self.bike)
            self.finished = True
            self.completed_at = datetime.now()

    def handle_issue_report(user, bikes, repairs_list):
        available_bikes = [bike for bike in bikes if bike.status != 'w naprawie']
        if not available_bikes:
            print("Brak rowerów do zgłoszenia.")
            return
        print("\nRowery dostępne do zgłoszenia:")
        for bike in available_bikes:
            status = bike.status
            station = bike.current_station.name if bike.current_station else "poza stacją"
            print(f"ID: {bike.bike_id}, Typ: {bike.type}, Status: {status}, Lokalizacja: {station}")
        try:
            bike_id = int(input("Podaj ID roweru do zgłoszenia usterki: "))
            bike = next((b for b in available_bikes if b.bike_id == bike_id), None)
            if bike:
                repair = Repair(bike, user)
                repairs_list.append(repair)
                print(f"Zgłoszono usterkę roweru {bike.bike_id}.")
            else:
                print("Nie znaleziono roweru.")
        except ValueError:
            print("Nieprawidłowy numer.")




class User:
    count = 1

    def __init__(self, name):
        self.user_id = User.count
        User.count += 1
        self.name = name
        self.active_rentals = []
        self.reservations = []

    def wypozycz_bike(self, bike, station):
        if bike.status != 'dostępny':
            raise Exception("Rower niedostępny.")
        rental = Rental(self, bike, station)
        self.active_rentals.append(rental)
        station.remove_bike(bike)
        bike.change_status('wypożyczony')
        return rental

    def zwroc_bike(self, rental, station):
        if rental not in self.active_rentals or not rental.active:
            raise Exception("Nieprawidłowe wypożyczenie.")
        rental.end_rental(station)
        self.active_rentals.remove(rental)

    def aktywne_rezerwacje(self):
        return [r for r in self.reservations if r.is_active()]

    def rent_bike_from_station(self, station):
        available = station.get_available_bikes()
        if not available:
            print("Brak dostępnych rowerów.")
            return None

        for reservation in self.aktywne_rezerwacje():
            for bike in available:
                if bike.type == reservation.bike_type:
                    rental = self.wypozycz_bike(bike, station)
                    reservation.fulfill()
                    print(f"Zrealizowano rezerwację: {bike.bike_id}")
                    print(f"Rower {bike.bike_id} został wypożyczony.")
                    return rental

        station.print_available_bikes()
        try:
            chosen_id = int(input("Podaj ID roweru do wypożyczenia: "))
            bike = next((b for b in available if b.bike_id == chosen_id), None)
            if bike:
                rental=self.wypozycz_bike(bike, station)
                print(f"Rower {bike.bike_id} został wypożyczony.")
                return rental
                
        except ValueError:
            pass
        print("Nieprawidłowy wybór.")
        return None

    def return_bike_to_station(self, station):
        active = [r for r in self.active_rentals if r.active]
        if not active:
            print("Brak aktywnych wypożyczeń.")
            return
        for i, rental in enumerate(active):
            print(f"{i+1}. Rower {rental.bike.bike_id}, stacja: {rental.start_station.name}")
        try:
            choice = int(input("Wybierz numer wypożyczenia: ")) - 1
            self.zwroc_bike(active[choice], station)
            print("Zwrot roweru zakończony sukcesem!")
        except Exception:
            print("Błąd przy zwrocie.")

    def make_reservation(self, station):
        bike_type = input("Podaj typ roweru do rezerwacji: ")
        teraz = datetime.now()
        wygasa = teraz + timedelta(minutes=15)
        reservation = Reservation(self, station, bike_type, teraz, wygasa)
        self.reservations.append(reservation)
        print(f"Zrobiono rezerwację: {reservation}")
        return reservation

    def print_reservations(self):
        if not self.reservations:
            print("Brak rezerwacji.")
            return
        for r in self.reservations:
            print(f"{r} - Status: {'aktywna' if r.is_active() else 'nieaktywna'}")

    def report_issue(self, bike, simulation):
        if bike.status == 'w naprawie':
            print("Rower już zgłoszony do naprawy.")
            return
        repair = Repair(bike, self)
        simulation.repairs.append(repair)
        print(f"Zgłoszono usterkę roweru {bike.bike_id}.")

from IPython.display import clear_output
    
class Simulation:
    def __init__(self):
        with open("stations.json", "r", encoding="utf-8") as f:
            station_data = json.load(f)
        self.stations = [Station(s["name"], s["location"], s["capacity"]) for s in station_data]

        with open("bikes.json", "r", encoding="utf-8") as f:
            bike_data = json.load(f)
        self.bikes = []
        for b in bike_data:
            bike = Bike(b["type"])
            self.bikes.append(bike)
            station_index = b["station_index"]
            self.stations[station_index].add_bike(bike)

        self.users = []
        self.rentals = []
        self.reservations = []
        self.repairs = []

    def run(self):
        print("=== SYSTEM WYPOŻYCZALNI ROWERÓW ===")

        while True:
            name = input("\nPodaj swoje imię (lub 'exit', aby zakończyć symulację): ").strip()
            if name.lower() == "exit":
                print("Zakończono działanie systemu.")
                break

            user = next((u for u in self.users if u.name == name), None)
            if not user:
                user = User(name)
                self.users.append(user)
                print(f"Utworzono nowego użytkownika: {user.name}")
            else:
                print(f"Witaj ponownie, {user.name}!")

            self.run_user_session(user)

    def run_user_session(self, user):
        while True:
            print(f"\nMENU - {user.name}:")
            print("1. Pokaż rowery")
            print("2. Wypożycz rower")
            print("3. Zwróć rower")
            print("4. Moje rezerwacje")
            print("5. Zrób rezerwację")
            print("6. Zgłoś usterkę")
            print("7. Wyloguj się")

            choice = input("Wybierz opcję: ")

            try:
                if choice == "1":
                    self.choose_station().print_available_bikes()
                elif choice == "2":
                    station = self.choose_station()
                    rental = user.rent_bike_from_station(station)
                    if rental:
                        self.rentals.append(rental)
                elif choice == "3":
                    user.return_bike_to_station(self.choose_station())
                elif choice == "4":
                    user.print_reservations()
                elif choice == "5":
                    reservation = user.make_reservation(self.choose_station())
                    self.reservations.append(reservation)
                elif choice == "6":
                    
                    Repair.handle_issue_report(user, self.bikes, self.repairs)

                elif choice == "7":
                    clear_output(wait=True)
                    print(f"Wylogowano {user.name}.")
                    
                    break
                else:
                    print("Nieprawidłowa opcja.")
            except Exception as e:
                print("Błąd:", e)

            self.process_repairs()

    def choose_station(self):
        for i, s in enumerate(self.stations):
            print(f"{i+1}. {s.name} ({s.location})")
        idx = int(input("Wybierz numer stacji: ")) - 1
        return self.stations[idx]

    def process_repairs(self):
        for repair in self.repairs:
            if not repair.finished and repair.is_done():
                # naprawione rowery wracają do pierwszej stacji
                repair.complete(self.stations[0])
                print(f"Rower {repair.bike.bike_id} naprawiony i zwrócony na stację {self.stations[0].name}.")


class Report:
    def __init__(self, users, rentals, stations):
        self.users = users
        self.rentals = rentals
        self.stations = stations


    def generate(self):
        print("\n=== RAPORT KOŃCOWY ===")

        print(f"Liczba użytkowników: {len(self.users)}")

        if not self.rentals:
            print("Brak wypożyczeń.")
            return

        # Najczęściej wypożyczany rower (po ID lub typie)
        bike_counter = Counter(rental.bike.bike_id for rental in self.rentals)
        most_common_bike = bike_counter.most_common(1)[0]
        print(f"Najczęściej wypożyczany rower: ID {most_common_bike[0]} ({most_common_bike[1]} razy)")

        # Najczęściej wybierana stacja startowa
        start_station_counter = Counter(rental.start_station.name for rental in self.rentals)
        most_common_start = start_station_counter.most_common(1)[0]
        print(f"Najczęściej wypożyczano ze stacji: {most_common_start[0]} ({most_common_start[1]} razy)")

        # Najczęściej wybierana stacja końcowa
        end_station_counter = Counter(rental.end_station.name for rental in self.rentals if rental.end_station)
        if end_station_counter:
            most_common_end = end_station_counter.most_common(1)[0]
            print(f"Najczęściej zwracano na stację: {most_common_end[0]} ({most_common_end[1]} razy)")
        else:
            print("Brak danych o zwrotach.")

        bike_type_counter = Counter(rental.bike.type for rental in self.rentals)
        top_type = bike_type_counter.most_common(1)[0]
        print(f"Najpopularniejszy typ roweru: {top_type[0]} ({top_type[1]} razy)")
        
# Uruchomienie systemu
if __name__ == "__main__":
    sim = Simulation()
    sim.run()


    report = Report(sim.users, sim.rentals, sim.stations)
    report.generate()

=== SYSTEM WYPOŻYCZALNI ROWERÓW ===
